# Camera Calibration




In [2]:
import os
import subprocess
import numpy as np
import imageio

## Let's start with a camera calibration matrix available on the web.. so we can compare results..


Downloading images and datapoints files

In [3]:
url_images = "http://research.microsoft.com/en-us/um/people/zhang/Calib/Calibration/CalibIm{}.gif"
url_data_pts = "http://research.microsoft.com/en-us/um/people/zhang/Calib/Calibration/data{}.txt"


data_path = './data/'
data = {}
images_filename = []
pts_fname = []
for num in range(1, 6):
    image_url = url_images.format(num)
    pts_url = url_data_pts.format(num)
    image_fname = os.path.join(data_path, os.path.basename(image_url))
    pts_fname = os.path.join(data_path, os.path.basename(pts_url))
    print(pts_fname)
    p = subprocess.Popen(['wget', image_url, '-O', image_fname])  # using wget simpler than urllib
    p.wait()
    
    p = subprocess.Popen(['wget', pts_url, '-O', pts_fname])  # using wget simpler than urllib
    p.wait()
    
    
    data[num] = {}
    data[num]['image_fname'] = image_fname
    data[num]['pts_fname'] = pts_fname
    
    
    pts = np.genfromtxt(data[num]['pts_fname'])
    pts = pts.reshape((256, 2))
    data[num]['pts'] = pts
    

    

./data/data1.txt
./data/data2.txt
./data/data3.txt
./data/data4.txt
./data/data5.txt


now we have a dictionnary with each image file associated to data points in  a textfile
data

In [1]:
data

NameError: name 'data' is not defined

# Camera calibration equations:

A 2D point in image coordinate (augmented to 3rd):  $m = [u, v, 1]^T $. 
A 3D point $M = [X, Y, Z, 1]^T$


$$
s * m = A [R \quad t] M
$$ 


with:
$ [R \quad t] = [ R \, |\, \boldsymbol{t}] = 
\left[ \begin{array}{ccc|c} 
r_{1,1} & r_{1,2} & r_{1,3} & t_1 \\
r_{2,1} & r_{2,2} & r_{2,3} & t_2 \\
r_{3,1} & r_{3,2} & r_{3,3} & t_3 \\
\end{array} \right]$

s a scale factor.
A the intrinsic camera matrix:

$$ A = \begin{bmatrix}\alpha & \gamma & u_0 \\ 0 & \beta & v_0  \\ 0 & 0 & 1
\end{bmatrix}
$$ 

We assume the object plane to be on Z = 0.

So we have:

$$ s * [u, v, 1]^T = A * [r_1 \quad r_2 \quad r_3 \quad t] * \begin{bmatrix}X\\Y\\0\\1\end{bmatrix}
$$

$$
= A * [r_1 \quad r_2  \quad t] * \begin{bmatrix}X\\Y\\Z\end{bmatrix}
$$

source: microsoftPaper



Constraints: r_1 and r_2 are orthonormal

$$ h_1^T * A^{-T} * A^-1 * h_2 = 0$$

$$ h_1^T * A^{-T} * A^-1 * 



we want to minimize :


$ \sum_{i}(m_i - \hat{m_i})^T  \Lambda^{-1}_{m_i} ({m_i - \hat{m_i}}) $

with $\Lambda_{m_i} = \sigma^2 * I $

and $\hat{m_i} =  (1 / ( h_3^T M_i)) * h_1^T M_i / (h_2^T Mi))$

Moui en fait plus simple.. 

on va minimiser : 
$ \sum_{i=1}^n \sum_{j=1}^m  \lvert\lvert ( m_{ij} - \hat{m_{ij}}) \rvert\rvert^2 $

In [ ]:
from mystic.models import AbstractFunction
class Rosenbrock(AbstractFunction):
    def __init__(self, ndim=2): # is n-dimensional (n=2 in ref)
        AbstractFunction.__init__(self, ndim=ndim)
        return

    def function(self, coeffs):
        """evaluates an N-dimensional Rosenbrock saddle for a list of coeffs
f(x) = \sum_(i=0)^(N-2) 100*(x_(i+1) - x_(i)^(2))^(2) + (1 - x_(i))^(2)
Inspect with mystic_model_plotter using::
    mystic.models.rosen -b "-3:3:.1, -1:5:.1, 1" -d -x 1
The minimum is f(x)=0.0 at x_i=1.0 for all i"""
        x = [1]*2 # ensure that there are 2 coefficients
        x[:len(coeffs)]=coeffs
        x = asarray(x) #XXX: must be a numpy.array
        return numpysum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)#,axis=0)

In [12]:
from mystic.solvers import fmin
from mystic.models import rosen
x0 = [1, 555, 3]
%qtconsole
solution = fmin(rosen,x0)
print(solution)

Optimization terminated successfully.
         Current function value: 1.689378
         Iterations: 121
         Function evaluations: 229
[ 1.48446709  2.20556484  4.86753823]


### Model optimization

In [ ]:
Model of the camera calibration function

In [9]:
import mystic

from mystic.forward_model import CostFactory
from numpy import array, sum, poly1d, random
import numpy as np

# build the forward model
def ForwardPolyFactory(coeff):
    """generate a 1-D polynomial instance from a list of coefficients"""
    from numpy import poly1d
    return poly1d(coeff)

%qtconsole

ForwardFactory = ForwardPolyFactory(m)

F = CostFactory()
costmetric = lambda x: np.sum(x*x)
F.addModel(ForwardFactory, name='pinhole', inputs=3)
#costunction = F.getCostFunction(evalpts=pts, )

NameError: name 'm' is not defined